In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# 里程计运动模型

技术上说，里程计信息就是传感器测量，而不是控制。为了建立作为测量的里程计模型，产生的贝叶斯滤波必须包括作为状态变量的实际速度——这会增加状态空间的维度。为了保持状态空间比较小，通常把里程计数据认为是控制信号。

里程计模型使用相对运动信息（Relative Motion Information），该信息由机器人内部里程计测量。在时间间隔$(t-1, t]$内，机器人从位姿$x_{t-1}$前进到位姿$x_t$。里程计反馈了

![](./img/odometry.jpg)